# Assignment 3 
### Yang Zhang  1726414


## Q1 Model Construction and Solving 

### 1.1 Solve "Inst1"

1.1.1 Import needed packages

In [1]:
import docplex
from docplex.mp.model import Model
import pandas as pd
import math
import numpy as np

1.1.2. Import Data inst1

In [2]:
# input data "inst1.xlsx"
data = pd.read_excel("inst1.xlsx")
U_cp = data.iloc[1:6,1:5] # time for unloading u_cp
ship_catagories = data.iloc[1:9,6:8] # ship catagories
ship_catagories.columns = ["ship","catagories"]
ship_arrival = data.iloc[1:9,9:11] 
ship_arrival.columns = ["ship","arrival_time"]

In [3]:
# merge the two dataframe ship_catagories and ship_arrival
ship_data = pd.concat([ship_catagories, ship_arrival.iloc[:,1]], axis=1)

ship = ship_arrival.iloc[:,0].tolist() # list of ships (sorted in arrival time)
S = list(range(1,len(ship)+1)) # represents the arrival order of each ship s
a = ship_data.iloc[:,2].tolist() # arrival time of each ship in sequence a_s
P = list(range(1,5)) # list of stations

# define a parameter U_sp to represent the unloading time of each ship in each station 
U = [] # change U_cp into U_sp
for i in range(0,len(ship)):
    x = ship_data.iloc[i,1]
    U.append(U_cp.iloc[x-1,:].tolist())

1.1.3. Model construnction

In [4]:
# create model
model = Model(name="Jetty Scheduling inst1", log_output=True)

1.1.4. Define decision variables

In [5]:
# decision variables
T = model.integer_var_matrix(S,P, name = "startTime") # The start time when ships on position k in the order reach station p (T[k,p])
D = model.integer_var_dict(S, name = "leaveTime") # The Leave time when ships on position k leaves the entire jetty (D[k])
Z = model.binary_var_cube(S,S,P, name = "order2") # Binary, equal to 1 when ship s is in order k and unloads in station p (Z[s,k,p])
O = model.binary_var_matrix(S,S, name = "order") # Binary, equal to 1 when ship s is in order k (O[s,k])

1.1.5. Add Constraints

In [6]:
# Model building based on "Permutation Flow Shop Scheduling"

# Each ship corresponds to only one position in the order, each ship unload at only one station, one station can only serve one ship.
for s in S:
    model.add_constraint(model.sum(Z[s,k,p] for k in S  for p in P) == 1)
# Similarly, Each position corresponds to only one ship and one station.
for k in S:
    model.add_constraint(model.sum(Z[s,k,p] for s in S  for p in P) == 1)
    
# Each ship is assigned to a position in a sequenece.
for s in S:
    model.add_constraint(sum(O[s,k] for k in S) == 1)
for k in S:
    model.add_constraint(sum(O[s,k] for s in S) == 1)
    
# link two binary variables O and Z
for s in S:
    for k in S:
        model.add_constraint(model.sum(Z[s,k,p] for p in P) == O[s,k])

# For ship in each position, it can enter the next station only when finish unloading at the current station (or just passing the station)
for k in S:
    for p in range(1,len(P)):
        model.add_constraint(T[k,p+1] >= T[k,p] + model.sum(Z[s,k,p]*U[s-1][p-1] for s in S) + 1 - model.sum(Z[s,k,p] for s in S))
        
# The time for a ship to leave the jetty should not be earlier than the start time entering the last station (or passing through).
for k in S:
    model.add_constraint(T[k,len(P)] + model.sum(Z[s,k,len(P)]*U[s-1][len(P)-1] for s in S) + 1 - model.sum(Z[s,k,len(P)] for s in S) <= D[k])
    
# For each station, the ship in the next position only enters after the former ship left. 
# Therefore, the entering time of the latter ship position should be larger than the leaving time of the former ship position.
for p in range(1,len(P)):
    for k in range(1,len(S)):
        model.add_constraint(T[k+1,p] >= T[k,p+1])
for k in range(1,len(S)):
    model.add_constraint(T[k+1,len(P)] >= D[k])

# T is non-negative
for k in S:
    for p in P:
        model.add_constraint(T[k,p] >= 0)

# A ship can only enter the jetty when it arrived at the jetty. 
# The entering time should be larger than the arrival time.
for k in S:
    model.add_constraint(T[k,1] >= model.sum(O[s,k]*a[s-1] for s in S))

1.1.6. objective function

In [7]:
# Objective function
# minimize the total time of each ship staying at the waiting area and the jetty.
TotalTime = model.sum(D[k] - a[k-1] for k in S)
model.minimize(TotalTime)

1.1.7. Solve the model and show details of the solving process

In [8]:
# Solve the model
model.solve()
# model.report()
model.print_solution()

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 40127.000000 after 0.00 sec. (0.06 ticks)
Tried aggregator 3 times.
MIP Presolve eliminated 39 rows and 52 columns.
MIP Presolve modified 79 coefficients.
Aggregator did 64 substitutions.
Reduced MIP has 93 rows, 244 columns, and 1023 nonzeros.
Reduced MIP has 224 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (2.89 ticks)
Probing fixed 0 vars, tightened 1 bounds.
Probing time = 0.00 sec. (1.13 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 2 rows and 2 columns.
Reduced MIP has 91 rows, 242 columns, and 1019 nonzeros.
Reduced MIP has 222 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.87 ticks)
Probing time = 0.00 sec. (1.10 ticks)
Clique table members: 460.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministi

In [9]:
# Print details of the solving process
print(model.solve_details) # prints details of the problem, e.g. the solve status, the type of the problem, ...
print(model.report()) # ptint if the model was solved and which objective function value results
print("The number of constraints is", model.number_of_constraints) # returns the number of constraints that were added to the model
print("The solve status is", model.get_solve_status()) # was the model solved to optimality?
print("The objective value is", model.objective_value) # print the objective function value
print("The time limit was fixed to", model.get_time_limit()) # print the chosen time limit
print("The optimality gap was fixed to  a tolerance of", model.parameters.mip.tolerances.mipgap.get()) # print the chosen ooptimality gap
print("The CPU time is", model.solve_details.time, "seconds") # print the CPU time for the solve process

status  = integer optimal solution
time    = 0.218 s.
problem = MILP
gap     = 0%

* model Jetty Scheduling inst1 solved with objective = 64.000
None
The number of constraints is 196
The solve status is JobSolveStatus.OPTIMAL_SOLUTION
The objective value is 64.0
The time limit was fixed to 1e+75
The optimality gap was fixed to  a tolerance of 0.0001
The CPU time is 0.21799999999348074 seconds


1.1.8. output the results

In [10]:
# Show the ship's order
col = ["ship number","ship name","order","unload at sation"]
res_Order1 = pd.DataFrame(columns = col)
for s in S:
    res_Order1.loc[s-1,"ship number"] = s
    res_Order1.loc[s-1,"ship name"] = ship[s-1]    
    res_Order1.loc[s-1,"order"] = round(sum(k*O[s,k].solution_value for k in S)) 
    
    res_Order1.loc[s-1,"unload at sation"] = round(sum(p*Z[s,res_Order1.iloc[s-1,2],p].solution_value for p in P))
res_Order1

,ship number,ship name,order,unload at sation
0,1,Exxon,2,1
1,2,Nina,7,1
2,3,Moers,4,4
3,4,Tornado,3,4
4,5,Kabul,1,4
5,6,Clausthal,8,2
6,7,Opladen,5,1
7,8,Herta,6,1


In [11]:
# Show the solution table
# create a dataframe to store results
timeSpan = round(T[len(S),len(P)].solution_value)
data = np.zeros([len(P),timeSpan])
row = ["time{}".format(t) for t in range(1,timeSpan+1)]
col = ["P{}".format(p) for p in P]
res_T1 = pd.DataFrame(data, col, row)

# Since a ship is always in a station after entering the jetty (represented with ship's order number)
for k in S:
    for p in P:
        t = round(T[k,p].solution_value)
        # when a ship is moving between all stations
        if p < len(P):
            while round(T[k,p+1].solution_value) > t:
                res_T1.iloc[p-1,t-1] = round(k)
                t += 1
        else: # when a ship is ready to leave the last station
             while round(D[k].solution_value) > t:
                res_T1.iloc[p-1,t-1] = round(k)
                t += 1           

# replace ship's order number with correspondent ship name, if there is no ship, use symbol "-" 
for t in range(1,timeSpan+1):
    for p in P:
        if res_T1.iloc[p-1,t-1] != 0:
            k = res_T1.iloc[p-1,t-1]
            j = round(sum(s*O[s,k].solution_value for s in S)) # ship number
            res_T1.iloc[p-1,t-1] = ship[j-1] # ship name
        else :
            res_T1.iloc[p-1,t-1] = "-"

res_T1

,time1,time2,time3,time4,time5,time6,time7,time8,time9,time10,...,time12,time13,time14,time15,time16,time17,time18,time19,time20,time21
P1,Kabul,Exxon,Exxon,Tornado,Moers,Opladen,Opladen,Herta,Herta,Nina,...,Nina,Clausthal,-,-,-,-,-,-,-,-
P2,-,Kabul,-,Exxon,Tornado,Moers,-,Opladen,-,Herta,...,-,Nina,Clausthal,Clausthal,Clausthal,Clausthal,Clausthal,Clausthal,-,-
P3,-,-,Kabul,-,Exxon,Tornado,Moers,-,Opladen,-,...,-,-,Nina,-,-,-,-,-,Clausthal,-
P4,-,-,-,Kabul,-,Exxon,Tornado,Moers,Moers,Opladen,...,Herta,-,-,Nina,-,-,-,-,-,Clausthal


1.1.9 Export to excel

In [12]:
# Export Inst1 results to Excel 
from pandas import ExcelWriter
with ExcelWriter("Assignment 3 Inst1 result .xlsx") as writer:
    res_Order1.to_excel(writer,index=False,sheet_name="Order and Unload",na_rep='NaN')
    res_T1.to_excel(writer,sheet_name="Flow plan",na_rep='NaN')
    #Adjust column widths
    for column in res_Order1:
        column_width = max(res_Order1[column].astype(str).map(len).max(), len(column))
        col_idx = res_Order1.columns.get_loc(column)
        writer.sheets['Order and Unload'].set_column(col_idx, col_idx, column_width+2)
    writer.save()
    res_Order1.to_excel(writer,index=False,sheet_name="Order and Unload",na_rep='NaN')

# -----------------------------------------Inst 1 solved-------------------------------------------------

### 1.2 Solve "Inst2"

In [13]:
# Data of Jetty Scheduling inst2
# create model

model = Model(name="Jetty Scheduling inst2", log_output=True)

# input data "inst2.xlsx"
data = pd.read_excel("inst2.xlsx")
U_cp = data.iloc[1:6,1:5] # time for unloading u_cp
ship_catagories = data.iloc[1:11,6:8] # ship catagories
ship_catagories.columns = ["ship","catagories"]
ship_arrival = data.iloc[1:11,9:11] 
ship_arrival.columns = ["ship","arrival_time"]

In [14]:
# merge the two dataframe ship_catagories and ship_arrival
ship_data = pd.concat([ship_catagories, ship_arrival.iloc[:,1]], axis=1)

ship = ship_arrival.iloc[:,0].tolist() # list of ships (sorted in arrival time)
S = list(range(1,len(ship)+1)) # represents the arrival order of each ship s
a = ship_data.iloc[:,2].tolist() # arrival time of each ship in sequence a_s
P = list(range(1,5)) # list of stations

# define a parameter U_sp to represent the unloading time of each ship in each station 
U = [] # change U_cp into U_sp
for i in range(0,len(ship)):
    x = ship_data.iloc[i,1]
    U.append(U_cp.iloc[x-1,:].tolist())

In [15]:
# decision variables
T = model.integer_var_matrix(S,P, name = "startTime") # The start time when ships on position k in the order reach station p (T[k,p])
D = model.integer_var_dict(S, name = "leaveTime") # The Leave time when ships on position k leaves the entire jetty (D[k])
Z = model.binary_var_cube(S,S,P, name = "order2") # Binary, equal to 1 when ship s is in order k and unloads in station p (Z[s,k,p])
O = model.binary_var_matrix(S,S, name = "order") # Binary, equal to 1 when ship s is in order k (O[s,k])

In [16]:
# Model building based on "Permutation Flow Shop Scheduling"

# Each ship corresponds to only one position in the order, each ship unload at only one station, one station can only serve one ship.
for s in S:
    model.add_constraint(model.sum(Z[s,k,p] for k in S  for p in P) == 1)
# Similarly, Each position corresponds to only one ship and one station.
for k in S:
    model.add_constraint(model.sum(Z[s,k,p] for s in S  for p in P) == 1)
    
# Each ship is assigned to a position in a sequenece.
for s in S:
    model.add_constraint(sum(O[s,k] for k in S) == 1)
for k in S:
    model.add_constraint(sum(O[s,k] for s in S) == 1)
    
# link two binary variables O and Z
for s in S:
    for k in S:
        model.add_constraint(model.sum(Z[s,k,p] for p in P) == O[s,k])

# For ship in each position, it can enter the next station only when finish unloading at the current station (or just passing the station)
for k in S:
    for p in range(1,len(P)):
        model.add_constraint(T[k,p+1] >= T[k,p] + model.sum(Z[s,k,p]*U[s-1][p-1] for s in S) + 1 - model.sum(Z[s,k,p] for s in S))
        
# The time for a ship to leave the jetty should not be earlier than the start time entering the last station (or passing through).
for k in S:
    model.add_constraint(T[k,len(P)] + model.sum(Z[s,k,len(P)]*U[s-1][len(P)-1] for s in S) + 1 - model.sum(Z[s,k,len(P)] for s in S) <= D[k])
    
# For each station, the ship in the next position only enters after the former ship left. 
# Therefore, the entering time of the latter ship position should be larger than the leaving time of the former ship position.
for p in range(1,len(P)):
    for k in range(1,len(S)):
        model.add_constraint(T[k+1,p] >= T[k,p+1])
for k in range(1,len(S)):
    model.add_constraint(T[k+1,len(P)] >= D[k])

# T is non-negative
for k in S:
    for p in P:
        model.add_constraint(T[k,p] >= 0)

# A ship can only enter the jetty when it arrived at the jetty. 
# The entering time should be larger than the arrival time.
for k in S:
    model.add_constraint(T[k,1] >= model.sum(O[s,k]*a[s-1] for s in S))

In [17]:
# Objective function
# minimize the total time of each ship staying at the waiting area and the jetty.
TotalTime = model.sum(D[k] - a[k-1] for k in S)
model.minimize(TotalTime)

In [18]:
# Solve the model
model.solve()
# model.report()
model.print_solution()

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 240103.000000 after 0.00 sec. (0.09 ticks)
Tried aggregator 3 times.
MIP Presolve eliminated 43 rows and 62 columns.
MIP Presolve modified 152 coefficients.
Aggregator did 71 substitutions.
Reduced MIP has 152 rows, 417 columns, and 1812 nonzeros.
Reduced MIP has 390 binaries, 27 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (4.01 ticks)
Probing time = 0.00 sec. (2.79 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 6 rows and 6 columns.
MIP Presolve modified 18 coefficients.
Reduced MIP has 146 rows, 411 columns, and 1800 nonzeros.
Reduced MIP has 384 binaries, 27 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (2.16 ticks)
Probing time = 0.01 sec. (2.32 ticks)
Clique table members: 864.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: determinist

In [19]:
# Show the ship's order
col = ["ship number","ship name","order","unload at sation"]
res_Order2 = pd.DataFrame(columns = col)
for s in S:
    res_Order2.loc[s-1,"ship number"] = s
    res_Order2.loc[s-1,"ship name"] = ship[s-1]    
    res_Order2.loc[s-1,"order"] = round(sum(k*O[s,k].solution_value for k in S)) 
    
    res_Order2.loc[s-1,"unload at sation"] = round(sum(p*Z[s,res_Order2.iloc[s-1,2],p].solution_value for p in P))
res_Order2

,ship number,ship name,order,unload at sation
0,1,Natalie,7,3
1,2,Neptun,10,2
2,3,Duandra,1,4
3,4,Waldhof,5,4
4,5,Panega,9,1
5,6,Megara,6,1
6,7,Onassis,2,1
7,8,Hansa,8,2
8,9,Carina,3,4
9,10,Atlantic,4,4


In [20]:
# Show the solution table
# create a dataframe to store results
timeSpan = round(T[len(S),len(P)].solution_value)
data = np.zeros([len(P),timeSpan])
row = ["time{}".format(t) for t in range(1,timeSpan+1)]
col = ["P{}".format(p) for p in P]
res_T2 = pd.DataFrame(data, col, row)

# Since a ship is always in a station after entering the jetty (represented with ship's order number)
for k in S:
    for p in P:
        t = round(T[k,p].solution_value)
        # when a ship is moving between all stations
        if p < len(P):
            while round(T[k,p+1].solution_value) > t:
                res_T2.iloc[p-1,t-1] = round(k)
                t += 1
        else: # when a ship is ready to leave the last station
             while round(D[k].solution_value) > t:
                res_T2.iloc[p-1,t-1] = round(k)
                t += 1           

# replace ship's order number with correspondent ship name, if there is no ship, use symbol "-" 
for t in range(1,timeSpan+1):
    for p in P:
        if res_T2.iloc[p-1,t-1] != 0:
            k = res_T2.iloc[p-1,t-1]
            j = round(sum(s*O[s,k].solution_value for s in S)) # ship number
            res_T2.iloc[p-1,t-1] = ship[j-1] # ship name
        else :
            res_T2.iloc[p-1,t-1] = "-"

res_T2

,time1,time2,time3,time4,time5,time6,time7,time8,time9,time10,...,time18,time19,time20,time21,time22,time23,time24,time25,time26,time27
P1,-,Duandra,Onassis,Onassis,Carina,Atlantic,Waldhof,Megara,Megara,Natalie,...,Panega,Neptun,-,-,-,-,-,-,-,-
P2,-,-,Duandra,-,Onassis,Carina,Atlantic,Waldhof,Waldhof,Megara,...,Hansa,Panega,Neptun,Neptun,Neptun,Neptun,Neptun,Neptun,-,-
P3,-,-,-,Duandra,-,Onassis,Carina,Atlantic,-,Waldhof,...,Natalie,Hansa,Panega,-,-,-,-,-,Neptun,-
P4,-,-,-,-,Duandra,Duandra,Onassis,Carina,Atlantic,Atlantic,...,-,Natalie,Hansa,Panega,-,-,-,-,-,Neptun


In [21]:
# Export Inst2 results to Excel 
from pandas import ExcelWriter
with ExcelWriter("Assignment 3 Inst2 result .xlsx") as writer:
    res_Order2.to_excel(writer,index=False,sheet_name="Order and Unload",na_rep='NaN')
    res_T2.to_excel(writer,sheet_name="Flow plan",na_rep='NaN')
    #Adjust column widths
    for column in res_Order2:
        column_width = max(res_Order2[column].astype(str).map(len).max(), len(column))
        col_idx = res_Order2.columns.get_loc(column)
        writer.sheets['Order and Unload'].set_column(col_idx, col_idx, column_width+1)
    writer.save()
    res_Order2.to_excel(writer,index=False,sheet_name="Order and Unload",na_rep='NaN')

# ----------------------------------------Inst2 solved-----------------------------------------------------

# Q2 Decompostition

### Get lower bound from directly solving "inst3"

In [22]:
# Data of Jetty Scheduling inst3
# create model
model = Model(name="Jetty Scheduling inst3", log_output=True)

# input data
data = pd.read_excel("inst3.xlsx")
U_cp = data.iloc[1:6,1:5] # time for unloading u_cp
ship_catagories = data.iloc[1:21,6:8] # ship catagories
ship_catagories.columns = ["ship","catagories"]
ship_arrival = data.iloc[1:21,9:11] 
ship_arrival.columns = ["ship","arrival_time"]

In [23]:
# merge the two dataframe ship_catagories and ship_arrival
ship_data = pd.concat([ship_catagories, ship_arrival.iloc[:,1]], axis=1)
# sort ships by their arrival time
#ship_data.sort_values(by=["arrival_time"])

ship = ship_data.iloc[:,0].tolist() # list of ships
S = list(range(1,len(ship)+1)) # represents the arrival order of each ship s
a = ship_data.iloc[:,2].tolist() # arrival time of each ship in sequence a_s
P = list(range(1,5)) # list of stations

# define a parameter U_sp to represent the unloading time of each ship in each station 
U = [] # change U_cp into U_sp
for i in range(0,len(ship)):
    x = ship_data.iloc[i,1]
    U.append(U_cp.iloc[x-1,:].tolist())
    


In [24]:
T = model.integer_var_matrix(S,P, name = "startTime") # arrival time of each ship on each station
D = model.integer_var_dict(S, name = "leaveTime") # each ship's leaving timetion
Z = model.binary_var_cube(S,S,P, name = "order2") # ship s is in order k and unloads in station p
O = model.binary_var_matrix(S,S, name = "order") # ship s is in order k

In [25]:
# Model building based on "Permutation Flow Shop Scheduling"

# Each ship corresponds to only one position in the order, each ship unload at only one station, one station can only serve one ship.
for s in S:
    model.add_constraint(model.sum(Z[s,k,p] for k in S  for p in P) == 1)
# Similarly, Each position corresponds to only one ship and one station.
for k in S:
    model.add_constraint(model.sum(Z[s,k,p] for s in S  for p in P) == 1)
    
# Each ship is assigned to a position in a sequenece.
for s in S:
    model.add_constraint(sum(O[s,k] for k in S) == 1)
for k in S:
    model.add_constraint(sum(O[s,k] for s in S) == 1)
    
# link two binary variables O and Z
for s in S:
    for k in S:
        model.add_constraint(model.sum(Z[s,k,p] for p in P) == O[s,k])

# For ship in each position, it can enter the next station only when finish unloading at the current station (or just passing the station)
for k in S:
    for p in range(1,len(P)):
        model.add_constraint(T[k,p+1] >= T[k,p] + model.sum(Z[s,k,p]*U[s-1][p-1] for s in S) + 1 - model.sum(Z[s,k,p] for s in S))
        
# The time for a ship to leave the jetty should not be earlier than the start time entering the last station (or passing through).
for k in S:
    model.add_constraint(T[k,len(P)] + model.sum(Z[s,k,len(P)]*U[s-1][len(P)-1] for s in S) + 1 - model.sum(Z[s,k,len(P)] for s in S) <= D[k])
    
# For each station, the ship in the next position only enters after the former ship left. 
# Therefore, the entering time of the latter ship position should be larger than the leaving time of the former ship position.
for p in range(1,len(P)):
    for k in range(1,len(S)):
        model.add_constraint(T[k+1,p] >= T[k,p+1])
for k in range(1,len(S)):
    model.add_constraint(T[k+1,len(P)] >= D[k])

# T is non-negative
for k in S:
    for p in P:
        model.add_constraint(T[k,p] >= 0)

# A ship can only enter the jetty when it arrived at the jetty. 
# The entering time should be larger than the arrival time.
for k in S:
    model.add_constraint(T[k,1] >= model.sum(O[s,k]*a[s-1] for s in S))

In [26]:
# Objective function
# minimize the total time of each ship staying at the waiting area and the jetty.
TotalTime = model.sum(D[k] - a[k-1] for k in S)
model.minimize(TotalTime)

In [ ]:
# Solve the model
model.solve()
# model.report()
model.print_solution()

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 410470.000000 after 0.00 sec. (0.34 ticks)
Tried aggregator 3 times.
MIP Presolve eliminated 83 rows and 155 columns.
MIP Presolve modified 663 coefficients.
Aggregator did 250 substitutions.
Reduced MIP has 403 rows, 1695 columns, and 7283 nonzeros.
Reduced MIP has 1617 binaries, 78 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (16.11 ticks)
Probing time = 0.00 sec. (3.08 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 403 rows, 1695 columns, and 7283 nonzeros.
Reduced MIP has 1617 binaries, 78 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (5.86 ticks)
Probing time = 0.01 sec. (3.04 ticks)
Clique table members: 2913.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.08 sec. (46.69 t

In [ ]:
# Show the solution table
# create a dataframe to store results
timeSpan = round(T[len(S),len(P)].solution_value)
data = np.zeros([len(P),timeSpan])
row = ["time{}".format(t) for t in range(1,timeSpan+1)]
col = ["P{}".format(p) for p in P]
res_T = pd.DataFrame(data, col, row)

# Since a ship is always in a station after entering the jetty (represented with ship's order number)
for k in S:
    for p in P:
        t = round(T[k,p].solution_value)
        # when a ship is moving between all stations
        if p < len(P):
            while round(T[k,p+1].solution_value) > t:
                res_T.iloc[p-1,t-1] = round(k)
                t += 1
        else: # when a ship is ready to leave the last station
             while round(D[k].solution_value) > t:
                res_T.iloc[p-1,t-1] = round(k)
                t += 1           

# replace ship's order number with correspondent ship name, if there is no ship, use symbol "-" 
for t in range(1,timeSpan+1):
    for p in P:
        if res_T.iloc[p-1,t-1] != 0:
            k = res_T.iloc[p-1,t-1]
            j = round(sum(s*O[s,k].solution_value for s in S)) # ship number
            res_T.iloc[p-1,t-1] = ship[j-1] # ship name
        else :
            res_T.iloc[p-1,t-1] = "-"

res_T

In [ ]:
# Print details of the solving process
print(model.solve_details) # prints details of the problem, e.g. the solve status, the type of the problem, ...
print(model.report()) # ptint if the model was solved and which objective function value results
print("The number of constraints is", model.number_of_constraints) # returns the number of constraints that were added to the model
print("The solve status is", model.get_solve_status()) # was the model solved to optimality?
print("The objective value is", model.objective_value) # print the objective function value
print("The time limit was fixed to", model.get_time_limit()) # print the chosen time limit
print("The optimality gap was fixed to  a tolerance of", model.parameters.mip.tolerances.mipgap.get()) # print the chosen ooptimality gap
print("The CPU time is", model.solve_details.time, "seconds") # print the CPU time for the solve process

### Get upper bound using decomposition

2.1 input the data

In [ ]:
# Data of Jetty Scheduling inst3
# create model
model = Model(name="Jetty Scheduling inst3", log_output=True)

# input data
data = pd.read_excel("inst3.xlsx")
U_cp = data.iloc[1:6,1:5] # time for unloading u_cp
ship_catagories = data.iloc[1:21,6:8] # ship catagories
ship_catagories.columns = ["ship","catagories"]
ship_arrival = data.iloc[1:21,9:11] 
ship_arrival.columns = ["ship","arrival_time"]

In [ ]:
# merge the two dataframe ship_catagories and ship_arrival
ship_data = pd.concat([ship_catagories, ship_arrival.iloc[:,1]], axis=1)
# sort ships by their arrival time
ship_data = ship_data.sort_values(by=["arrival_time"])

ship = ship_data.iloc[:,0].tolist() # list of ships (sorted in arrival time)
S = list(range(1,len(ship)+1)) # represents the arrival order of each ship s
a = ship_data.iloc[:,2].tolist() # arrival time of each ship in sequence a_s
P = list(range(1,5)) # list of stations

# define a parameter U_sp to represent the unloading time of each ship in each station 
U = [] # change U_cp into U_sp
for i in range(0,len(ship)):
    x = ship_data.iloc[i,1]
    U.append(U_cp.iloc[x-1,:].tolist())
    


2.1 Define auxiliary parameters for rolling planning horizon

In [ ]:
timewindow = 10
fixed = 3 
overlap = timewindow - fixed
horizon = timewindow
num_iterations = math.ceil((len(S) - timewindow)/fixed) + 1 # calculate the number of iterations

2.2 Define decision variables

In [ ]:
T = model.integer_var_matrix(S,P, name = "startTime") # arrival time of each ship on each station
D = model.integer_var_dict(S, name = "leaveTime") # each ship's leaving timetion
Z = model.binary_var_cube(S,S,P, name = "order2") # ship s is in order k and unloads in station p
O = model.binary_var_matrix(S,S, name = "order") # ship s is in order k

2.3 Iteratively define the constraints, define the objective function, solve the model and shift the planning horizon

In [ ]:
CPU_time = 0
for i in range(1,num_iterations+1): # Iteratively do 
    
    #------------------------------------------------------------------------------------
    # Define constraints
    #------------------------------------------------------------------------------------
    # order of each ship
    for s in S:
        #if s <= horizon:
            model.add_constraint(model.sum(Z[s,k,p] for k in S  for p in P) == 1)
            model.add_constraint(model.sum(O[s,k] for k in S) == 1)
    for k in S:
        if k <= horizon:
            model.add_constraint(model.sum(Z[s,k,p] for s in S  for p in P) == 1)
            model.add_constraint(model.sum(O[s,k] for s in S) == 1)
    
    for s in S:
        #if s <= horizon:
            for k in S:
                if k <= horizon:
                    model.add_constraint(model.sum(Z[s,k,p] for p in P) == O[s,k])
    
    # For ship in each position, it can enter the next station 
    # only when finished unloading (or just passing) at the current station 
    for p in range(1,len(P)):
        for k in S:
            if k <= horizon:
                model.add_constraint(T[k,p+1] >= T[k,p] + model.sum(Z[s,k,p]*U[s-1][p-1] for s in S) + 1 - model.sum(Z[s,k,p] for s in S))
    # The time for a ship to enter the open sea should 
    # not be less than the time to enter the last station and pass through this station.
    for k in S:
        if k <= horizon:
            model.add_constraint(T[k,len(P)] + model.sum(Z[s,k,len(P)]*U[s-1][len(P)-1] for s in S) + 1 - model.sum(Z[s,k,len(P)] for s in S) <= D[k])
    # For each station, the next ship comes only when the former ship left. 
    # Therefore, the entering time of the latter ship should be larger than the leaving time of the former ship.
    for p in range(1,len(P)):
        for k in range(1,len(S)):
            if k < horizon: 
                model.add_constraint(T[k+1,p] >= T[k,p+1])
    for k in range(1,len(S)):
        if k < horizon:
            model.add_constraint(T[k+1,len(P)] >= D[k])
    
    # T is non-negative
    for p in P:
        for k in S:
            if k <= horizon:
                model.add_constraint(T[k,p] >= 0)
    for p in P:
        for k in S:
            if k <= horizon:
                model.add_constraint(D[k] >= 0)
    # A ship can only enter the jetty when it arrived at the jetty. 
    # The entering time should be larger than the arrival time.
    for k in S:
        model.add_constraint(T[k,1] >= model.sum(O[s,k]*a[s-1] for s in S))
    #------------------------------------------------------------------------------------
    # Define the objective function 
    #------------------------------------------------------------------------------------
    horizon2 = min(horizon+1,len(S)+1)
    TotalTime = model.sum(D[k] - a[k-1] for k in range(1,horizon2))
    model.minimize(TotalTime)
    
    #------------------------------------------------------------------------------------
    # Solve the model
    #------------------------------------------------------------------------------------
    
    #model.set_time_limit(5) # set a time limit
    #model.parameters.mip.tolerances.mipgap.set(0.05) # set a larger optimality gap
    model.solve()

    #------------------------------------------------------------------------------------
    # Print the results and additional information
    #------------------------------------------------------------------------------------
    
    print("The solve status is", model.get_solve_status()) # was the model solved to optimality?
    print("The objective value is", model.objective_value) # print the objective function value
    print("The time limit was fixed to", model.get_time_limit()) # print the chosen time limit
    print("The optimality gap was fixed to", model.parameters.mip.tolerances.mipgap.get()) # print the chosen optimality gap
    print("The CPU time is", model.solve_details.time, "seconds") # print the CPU time of the current iteration
    CPU_time = model.solve_details.time + CPU_time
    
    # order
    col = ["ship number","ship name","order"]
    res_Order = pd.DataFrame(columns = col)
    for s in S:
        res_Order.loc[s-1,"ship number"] = s
        res_Order.loc[s-1,"ship name"] = ship_data.iloc[s-1,0]    
        res_Order.loc[s-1,"order"] = round(sum(k*O[s,k].solution_value for k in range(1,horizon2))) 
    print(res_Order)
    
    #------------------------------------------------------------------------------------
    # Fix the decision variables of the current planning horizon
    #------------------------------------------------------------------------------------
    for s in S:
        # if s <= (horizon - overlap):
            for k in S:
                if k <= (horizon - overlap):
                    for p in P:
                        # for all periods until (horizon - overlap)
                        model.add_constraint(T[k,p] == T[k,p].solution_value)
                        model.add_constraint(D[k] == D[k].solution_value)
                        model.add_constraint(Z[s,k,p] == Z[s,k,p].solution_value)
                        model.add_constraint(O[s,k] == O[s,k].solution_value)
    
    #------------------------------------------------------------------------------------
    # Shift the planning horizon by (timewindow - overlap) periods
    #------------------------------------------------------------------------------------
    
    horizon = horizon + timewindow - overlap

print("The total CPU time is",CPU_time,"seconds.")